In [6]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/semcovici/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
processed_data_path = '../data/processed/'
file_format_tmt = processed_data_path + "{split}_r3_{target}_top_mentioned_timelines_processed.csv"
file_format_users = processed_data_path + 'r3_{target}_{split}_users_processed.csv' 

In [8]:
# ---------------------------------------------------
# 1. Define or load your Brazilian Portuguese stopwords
# ---------------------------------------------------
# Option A: Use nltk's 'portuguese' stopwords (they are not perfect for Brazilian Portuguese,
#           but it's a common starting point). You could also create your own custom list.

# Downloaded 'portuguese' stopwords
portuguese_stopwords = stopwords.words('portuguese')

# ---------------------------------------------------
# 2. Vectorize using TF-IDF
# ---------------------------------------------------

tfidf = TfidfVectorizer(
    stop_words=portuguese_stopwords,  # remove Brazilian Portuguese stopwords
    lowercase=True
)

In [9]:
target = 'ig'

df_tmt = pd.read_csv(
    file_format_tmt.format(target=target, split="train"), 
    sep=';', 
    encoding='utf-8-sig',
    index_col='User_ID'
).drop('Unnamed: 0',axis = 1)

df_tmt.head()

,Polarity,Texts
User_ID,,
r2_ig_1,against,PQP ESSE DORAMA É MUITO FOADA(Sassy GoGo(Cheer...
r2_ig_4,for,Golaço!!!!!!!!! # Manda geral do time principa...
r2_ig_7,against,"@ Amanhã vou aí, deixa pra terça # @ Bom dia b..."
r2_ig_8,for,3.4- O Centro de Coordenação da Operação está ...
r2_ig_10,for,"Me arrependi de excluir meu outro tt, agora ti..."


In [10]:
text_col = 'Texts'
label_col = 'Polarity'

X = tfidf.fit_transform(df_tmt[text_col])
y = df_tmt[label_col]  # Our target labels

# ---------------------------------------------------
# 3. Select top K best features using SelectKBest
# ---------------------------------------------------

# Choose how many features you want to keep, e.g., top 5
k = 50

selector = SelectKBest(chi2, k=k)
X_kbest = selector.fit_transform(X, y)

# ---------------------------------------------------
# 4. Examine the top features
# ---------------------------------------------------
feature_names = tfidf.get_feature_names_out()
selected_mask = selector.get_support()  # boolean mask of selected features
selected_features = [feature for feature, mask in zip(feature_names, selected_mask) if mask]

# Print out the most relevant features
print("Top {} selected features:".format(k))
for feat in selected_features:
    print(feat)


Top 5 selected features:
bolsonaro
covid
cpi
governo
kkk


In [11]:
df_users = pd.read_csv(
    file_format_users.format(target=target, split="train"), 
    sep=';', 
    encoding='utf-8-sig',
    index_col='User_ID'
).drop('Unnamed: 0',axis = 1)